In [122]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pymysql 

In [123]:
def get_columns(info_COLUMNS):
    lst=[]
    for i in info_COLUMNS:
        lst.append(i[0])
    return lst

In [124]:
# 用pymysql连接尝试
# 连接数据库
conn = pymysql.connect(
	host = '123.60.11.177',
	port = 3306,
	user = 'root',
	password = 'ncu@jw114',
	db = 'examArrange1',
	charset='utf8'
	)
cur = conn.cursor()
cur.execute("SELECT column_name FROM information_schema.columns WHERE (table_name = 'classroom_inf') AND (ordinal_position >= 1) order by ordinal_position")
classroom_inf_columns=get_columns(cur.fetchall())
cur.execute("SELECT * FROM classroom_inf")
classroom_inf=pd.DataFrame(cur.fetchall(),columns=classroom_inf_columns)

cur.execute("SELECT column_name FROM information_schema.columns WHERE (table_name = 'classs_inf') AND (ordinal_position >= 1) order by ordinal_position")
classs_inf_columns=get_columns(cur.fetchall())
cur.execute("SELECT * FROM classs_inf")
classs_inf=pd.DataFrame(cur.fetchall(),columns=classs_inf_columns)

cur.execute("SELECT column_name FROM information_schema.columns WHERE (table_name = 'dm_classroom_state') AND (ordinal_position >= 1) order by ordinal_position")
dm_classroom_state_columns=get_columns(cur.fetchall())
cur.execute("SELECT * FROM dm_classroom_state")
dm_classroom_state=pd.DataFrame(cur.fetchall(),columns=dm_classroom_state_columns)
#关闭数据库连接
conn.close()

# 选出公共课

In [125]:
#教室时间对
D=dm_classroom_state[dm_classroom_state.cr_name!='虚拟教室01'] #维护的一个笛卡儿排课表
dm_classroom_state[dm_classroom_state.cr_name!='虚拟教室01']
new_D=D[:]
#课表时间对
temp=pd.read_csv("./temp1.csv",names=["ci_course_no","test_time","ci_class_name","ci_student_number","ci_teacher_name"])
temp.insert(0,"index",temp.index.values)

# 两个输入数据

In [126]:
new_D #0是序号--后面可以用来存班级编号，
      #1位教室名称，2对应校区，3对应考试时间，4对应的是否能够使用，5对应是否安排考场

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
0,1,慧源楼101,前湖校区,1,1,None
1,2,慧源楼101,前湖校区,2,1,None
2,3,慧源楼101,前湖校区,3,1,None
3,4,慧源楼101,前湖校区,4,1,None
4,5,慧源楼101,前湖校区,5,1,None
...,...,...,...,...,...,...
2419,2420,慧源楼520,前湖校区,20,1,None
2420,2421,慧源楼520,前湖校区,21,1,None
2421,2422,慧源楼520,前湖校区,22,1,None
2422,2423,慧源楼520,前湖校区,23,1,None


In [127]:
temp #ci_nation_no班级编号，ci_course_no	课程名称（可不用），
     #test_time需要安排的考试时间，ci_class_name班级名称（可不用）

,index,ci_course_no,test_time,ci_class_name,ci_student_number,ci_teacher_name
0,0,550GL003,0,智能医学工程201班,30.0,李寅
1,1,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",124.0,廖清华
2,2,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",123.0,于天宝
3,3,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",125.0,陶永芊
4,4,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",125.0,邱天珍
...,...,...,...,...,...,...
114,114,590GL005,1,机械设计制造及其自动化207班,32.0,朱洪涛
115,115,590GL005,1,机械设计制造及其自动化[205-6]班,56.0,包艳
116,116,590GL005,1,机械设计制造及其自动化[201-2]班,60.0,肖承地
117,117,590GL005,1,机械设计制造及其自动化[203-4]班,59.0,赵朋龙


# 主要函数内容

In [128]:
#根据时间段找到所有教室
def get_test_time_room(test_time):
    test_time_room=D[(D.mti_no==test_time)&(D.cr_state==1)].cr_name.values
    return test_time_room
def fill_bz(room_name,test_time):
    new_D.loc[(new_D.cr_name==room_name)&(new_D.mti_no==test_time),"cr_state"]=0
    return 0
#当前的教室对应的班级index
def change_state_id(room_name,test_time,i):
    new_D.iloc[new_D[(new_D.cr_name==room_name)&(new_D.mti_no==test_time)].index[0],0]=i
    return 0
#判断当前考场是否安排
def classroom_isbz(cr_name,time_no):
    return (new_D[(new_D.cr_name==cr_name)&(new_D.mti_no==time_no)].cr_state==0).values==False
#判断当前时段的教室是否符合当前的课程
def student_num_fit(cr_names,cl_num):
    cr_num=classroom_inf[classroom_inf.cr_name==cr_names].cr_testseat.values
    if (cr_num<=(cl_num+10) and cr_num>=cl_num):
        return True
    else:
        return False

In [130]:
#功能描述：进行排考
#输入：教室时间笛卡儿乘积；班级编号与时间笛卡儿乘积
#输出：教室时间班级编号笛卡儿
def schedule_algorithm(new_D,temp):
    cannt_find_class_id=[]
    for i,cl_num,test_time in zip(temp.index,temp.ci_student_number,temp.test_time):
        test_time_room=get_test_time_room(test_time)                    
        for room_name in test_time_room:
            if classroom_isbz(room_name,test_time+1) and student_num_fit(room_name,cl_num):
                fill_bz(room_name,test_time+1)
                change_state_id(room_name,test_time+1,i)
                break
        else:
            cannt_find_class_id.append(i)
    return new_D,cannt_find_class_id

In [131]:
#检测并处理不在同一楼栋的课程
def same_course_is_same_campus(alldata,cannt_find_class_id):
    find_class_id=alldata[alldata.apply(lambda x:x[4]==0,axis=1)].state_id.values
    for i in find_class_id:
        find_class=temp[temp.apply(lambda x:x[0]==i,axis=1)].ci_course_no.values
        if find_class in cannt_find_class:
            alldata.loc[alldata.apply(lambda x:x[0]==i,axis=1),"cr_state"]=1
            cannt_find_class_id.append(i)

In [132]:
%%time
#alldata---所有考场的信息
#cannt_find_room---未安排的班级编号
#outdata-----已经安排好的考场情况
alldata,cannt_find_class_id=schedule_algorithm(new_D,temp)
# same_course_is_same_campus(alldata,cannt_find_class_id) 
outdata=alldata[alldata.cr_state==0]

Wall time: 7.24 s


In [134]:
# new_D.cr_state[new_D[(new_D.cr_name=="慧源楼101")&(new_D.mti_no==1)].index]=1
alldata

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
0,1,慧源楼101,前湖校区,1,1,None
1,42,慧源楼101,前湖校区,2,0,None
2,3,慧源楼101,前湖校区,3,1,None
3,4,慧源楼101,前湖校区,4,1,None
4,5,慧源楼101,前湖校区,5,1,None
...,...,...,...,...,...,...
2419,2420,慧源楼520,前湖校区,20,1,None
2420,2421,慧源楼520,前湖校区,21,1,None
2421,2422,慧源楼520,前湖校区,22,1,None
2422,2423,慧源楼520,前湖校区,23,1,None


In [135]:
outdata

,state_id,cr_name,cr_campus,mti_no,cr_state,bz
1,42,慧源楼101,前湖校区,2,0,None
25,99,慧源楼111,前湖校区,2,0,None
26,54,慧源楼111,前湖校区,3,0,None
49,103,慧源楼113,前湖校区,2,0,None
50,55,慧源楼113,前湖校区,3,0,None
73,104,慧源楼114,前湖校区,2,0,None
74,59,慧源楼114,前湖校区,3,0,None
97,105,慧源楼115,前湖校区,2,0,None
98,60,慧源楼115,前湖校区,3,0,None
121,106,慧源楼116,前湖校区,2,0,None


In [136]:
temp

,index,ci_course_no,test_time,ci_class_name,ci_student_number,ci_teacher_name
0,0,550GL003,0,智能医学工程201班,30.0,李寅
1,1,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",124.0,廖清华
2,2,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",123.0,于天宝
3,3,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",125.0,陶永芊
4,4,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",125.0,邱天珍
...,...,...,...,...,...,...
114,114,590GL005,1,机械设计制造及其自动化207班,32.0,朱洪涛
115,115,590GL005,1,机械设计制造及其自动化[205-6]班,56.0,包艳
116,116,590GL005,1,机械设计制造及其自动化[201-2]班,60.0,肖承地
117,117,590GL005,1,机械设计制造及其自动化[203-4]班,59.0,赵朋龙


In [137]:
print("成功排考班级",len(outdata))
print("未排考班级",len(cannt_find_class_id))
print("总共的排考班级",len(temp))

成功排考班级 36
未排考班级 83
总共的排考班级 119


In [138]:
classroom_inf[classroom_inf.cr_name=="慧源楼122"]


,cr_no,cr_name,cr_campus,cr_building,cr_dep,cr_special,cr_type,cr_expcenter,cr_2ndexpcenter,cr_testseat,cr_canuse,cr_writtenexam,cr_seatfix,cr_aircon,cr_remark
9,0,慧源楼122,前湖校区,慧源楼,None,否,多媒体教室,None,None,30,0,0,0,0,None


# 数据的导出

In [139]:
outdata=outdata.rename(columns={"state_id":"index"})
df=pd.merge(temp,outdata,how='inner',on=["index"]).iloc[:,[1,2,3,4,5,6]]
df

,ci_course_no,test_time,ci_class_name,ci_student_number,ci_teacher_name,cr_name
0,550GL005,1,药学[201-2]班,84.0,徐雪春,慧源楼135
1,550GL005,1,药学[203-4]班,80.0,廖清华,慧源楼101
2,550GL005,1,临床医学2011班,56.0,于天宝,慧源楼125
3,550GL005,1,"临床医学209班,临床医学2010班",97.0,陶永芊,慧源楼202B沉浸式教室
4,550GL005,1,"医学检验技术201班,基础医学201班",79.0,邱天珍,慧源楼201
5,550GL005,1,临床医学（留学生）201班,49.0,郭守晖,慧源楼126
6,550GL005,1,临床医学（留学生）202班,72.0,杨玉桃,慧源楼301
7,510GY017,2,"法学类[201-4]班,管理科学与工程类[201-4]班,机械设计制造及其自动化[201-7...",37.0,刘庆华,慧源楼111
8,510GY017,2,"法学类[201-4]班,管理科学与工程类[201-4]班,机械设计制造及其自动化[201-7...",38.0,邓琳娜,慧源楼113
9,510GY017,2,"水利水电工程[201-2]班,应用心理学[201-2]班,土木类[201-8]班,公共管理类...",39.0,黄霞,慧源楼238


In [54]:
df.to_csv("demo3.csv")

,ci_course_no,test_time,ci_course_name,ci_student_number,ci_teacher_name,cr_name,cr_campus
0,550GL003,0,智能医学工程201班,30.0,李寅,慧源楼101,前湖校区
1,Z521KB4009,0,音乐学（音乐教育）193班,14.0,万蓓,慧源楼111,前湖校区
2,Z521KB4009,0,音乐学（音乐表演）194班,12.0,黄海,慧源楼113,前湖校区
3,Z521KB4009,0,音乐学（音乐教育）193班,1.0,龚甜,慧源楼114,前湖校区
4,Z521KB4009,0,音乐学（音乐教育）192班,14.0,马振宇,慧源楼115,前湖校区
...,...,...,...,...,...,...,...
80,590GL005,1,机械设计制造及其自动化207班,32.0,朱洪涛,慧源楼218,前湖校区
81,590GL005,1,机械设计制造及其自动化[205-6]班,56.0,包艳,慧源楼226,前湖校区
82,590GL005,1,机械设计制造及其自动化[201-2]班,60.0,肖承地,慧源楼240,前湖校区
83,590GL005,1,机械设计制造及其自动化[203-4]班,59.0,赵朋龙,慧源楼342,前湖校区


In [47]:
#导出未排好班级考场数据
temp[temp.apply(lambda x:x[0] in cannt_find_room,axis=1)].iloc[:,1:]
# .to_csv("demo3_canfind.csv")

,ci_course_no,test_time,ci_class_name,ci_student_number,ci_teacher_name
1,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",124.0,廖清华
2,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",123.0,于天宝
3,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",125.0,陶永芊
4,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",125.0,邱天珍
5,550GL003,0,"水利水电工程[201-2]班,自动化[201-4]班,生物工程[201-2]班,食品科学与工...",106.0,唐玉超
...,...,...,...,...,...
109,590GL005,1,"车辆工程203班,能源与动力工程203班",63.0,胡贇
110,590GL005,1,能源与动力工程[201-2]班,91.0,谭刚平
111,590GL005,1,车辆工程[201-2]班,72.0,宋红滚
112,590GL005,1,材料成型及控制工程[201-2]班,77.0,胡莹


# 初始数据分析--与前面算法没直接关系

In [13]:
classroom_inf.iloc[:,9].value_counts()    #----考场座位分布情况

38     37
30     36
36      8
58      6
62      5
80      3
48      3
100     1
94      1
68      1
Name: 9, dtype: int64